In [1]:
import fxcmpy, socketio, engineio
print(fxcmpy.__version__)
TOKEN = "5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27"
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level="error")
con

1.3.2


In [9]:
import datetime as dt
from icecream import ic 
import socketio, engineio
from django.conf import settings

start = dt.datetime(2022,6,14)
end = dt.datetime(2022,6,16)
data = con.get_candles('EUR/USD', period='m1', start=start, end=end) #number=250 start=start, end=end
# data.to_csv("may.csv")
connection = con
data

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
date,,,,,,,,,
2022-06-14 00:00:00,1.04143,1.04137,1.04143,1.04114,1.04157,1.04151,1.04157,1.04127,148
2022-06-14 00:01:00,1.04136,1.04069,1.04136,1.04068,1.04151,1.04083,1.04151,1.04081,173
2022-06-14 00:02:00,1.04069,1.04045,1.04076,1.04045,1.04082,1.04060,1.04090,1.04059,211
2022-06-14 00:03:00,1.04045,1.04042,1.04077,1.04042,1.04059,1.04056,1.04089,1.04056,210
2022-06-14 00:04:00,1.04041,1.04053,1.04058,1.04031,1.04056,1.04066,1.04073,1.04045,173
...,...,...,...,...,...,...,...,...,...
2022-06-14 15:25:00,1.04185,1.04196,1.04210,1.04180,1.04197,1.04209,1.04222,1.04193,378
2022-06-14 15:26:00,1.04197,1.04206,1.04211,1.04197,1.04210,1.04218,1.04223,1.04209,264
2022-06-14 15:27:00,1.04205,1.04212,1.04225,1.04201,1.04218,1.04224,1.04238,1.04213,219


In [10]:
import django
django.setup()
import importlib
from datetime import datetime
import time, os, json
# import trading
# importlib.reload(trading)
# from trading import trading
# importlib.reload(trading)
# from trading.trading import *
from trading import models
importlib.reload(models)
from trading.models import Order

|ERROR|2022-06-13 03:49:45,265|Socket returns unknown error.


ServerError: Can not connect to FXCM Server.

In [ ]:
from datetime import datetime, date, time, timedelta
import time as gtime

def start_forex_trade():
    connection = Connection().connect()
    print(connection)
    print("Connection established successfully!")
    while datetime.today().isoweekday() in list(range(1,6)):
        try:
            today_date = datetime.today()
            start_time = time(0,10)
            end_time = time(20,49)
            start_trading = False
            if today_date.isoweekday() in list(range(1,6)):
                if today_date.isoweekday() == 1 and today_date.time() > start_time:
                    start_trading = True
                elif today_date.isoweekday() == 5 and today_date.time() > end_time:
                    start_trading = False
                else:
                    start_trading = True

                if start_trading:
                    gtime.sleep(60)
                    df = connection.get_candles('EUR/USD', period='m1', number=250)
                    gtime.sleep(1)
                    print(df.iloc[-1])
                    start_entry_trading(SMAStrategy3Level1, stoploss=0.0005, target=0.0008, df=df)
                    start_exit_trading(SMAExitLevel1, stoploss=0.0005, target=0.0008, df=df)
        except:
            continue
                
start_forex_trade()


None
Connection established successfully!


In [5]:
import datetime
from django.utils import timezone
# entry = FindEntry().find_entry(SMAStrategy3Level1, data, stoploss=0.0005, target=0.0008)
# order_book = OrderBook()
# ordr = Order.objects.first()

# ic(order_book.update_order(entry, "exit", dd))
# ic(order_book.last_order())
# ic(order_book.is_last_order_open())
# js_data = OrderBook.jsonize_entry(entry)

# # ordr._entry = js_data
# # ordr.save()
# # ordr._entry == js_data
# ordr._entry


In [216]:
def start_entry_trading(entry_strategy,  **kwargs):
    entry_signal = FindEntry().find_entry(entry_strategy, **kwargs)
    order_book = OrderBook()
    order_placed = False
    order = None
    if entry_signal:
        _entry_signal = next(pd.DataFrame(entry_signal).itertuples())
        last_order = order_book.last_order()
        AMOUNT = kwargs.get("AMOUNT", 5) # this is lot size
        pip_profit = kwargs.get("pip_profit", 0.004)
        LIMIT_PRICE = _entry_signal.entry_price + pip_profit if _entry_signal.entry_signal == "BUY" else _entry_signal.entry_price - pip_profit
        if last_order and order_book.is_last_order_open():
            last_order_df = next(pd.DataFrame(last_order.entry).itertuples())
            if last_order_df.entry_signal == _entry_signal.entry_signal and order_book.update_order(entry_signal):
                print("Entry signal found placing a new order")
                order = connection.create_entry_order(
                            symbol='EUR/USD',
                            is_buy=True if _entry_signal.entry_signal == "BUY" else False,
                            amount=AMOUNT,
                            order_type="Entry",
                            rate=_entry_signal.entry_price,
                            limit=LIMIT_PRICE,
                            is_in_pips=False,
                            time_in_force='GTC',
                        )
            elif last_order_df.entry_signal != _entry_signal.entry_signal:
                if order_book.update_order(entry_signal):
                    print(f"Send Order for {entry_signal}")
#                 Comment in below code when exit signal function is ready
                    order = connection.create_entry_order(
                            symbol='EUR/USD',
                            is_buy=True if _entry_signal.entry_signal == "BUY" else False,
                            amount=AMOUNT,
                            order_type="Entry",
                            rate=_entry_signal.entry_price,
                            limit=LIMIT_PRICE,
                            is_in_pips=False,
                            time_in_force='GTC',
                        )
        else:
            if order_book.update_order(entry_signal):
                print("Entry signal found placing a new order!....")
                order = connection.create_entry_order(
                            symbol='EUR/USD',
                            is_buy=True if _entry_signal.entry_signal == "BUY" else False,
                            amount=AMOUNT,
                            order_type="Entry",
                            rate=_entry_signal.entry_price,
                            limit=LIMIT_PRICE,
                            is_in_pips=False,
                            time_in_force='GTC',
                        )
        if order:
            if OrderBook.last_order():
                od = OrderBook.last_order()
                od.orderId = order.get_orderId()
                od.save()
                print("Order placed successfully!")
    else:
        print("no signal found :(!")
        

start_entry_trading(SMAStrategy3Level1, stoploss=0.0005, target=0.0008, df=data)
# od = OrderBook.last_order()
# od.order_time

/home/sonu/Desktop/Sonu/aff/env/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1418: RuntimeWarning: DateTimeField Order.order_time received a naive datetime (2022-06-06 08:15:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [208]:
import requests

def close_order(order_obj, exit_signal:list):
    if not order_obj.has_exit_signal and exit_signal != (0,0,0):
        order = connection.get_order(order_obj.orderId)
        tradeId = order.get_tradeId()
        if tradeId:
            request_data = {
                "trade_id": tradeId,
                "amount": order.get_associated_trade().get_amount(),
                "order_type": "AtMarket",
                "time_in_force": "FOK",
                "rate": exit_signal[0],
                "at_market": 0
            }
            print(request_data)
            res = requests.post('%s:443/%s' % (connection.trading_url, "trading/close_trade") ,headers=connection.request_headers, data=request_data)
            print(res.json())
        else:
            print("Order not created yet")


def start_exit_trading(exit_strategy, **kwargs):
    order_book = OrderBook()
    last_order = order_book.last_order()
    open_orders = [order for order in Order.objects.filter(order_time__date=datetime.today().date()) if not order.has_exit_signal]
    
    for order in open_orders:
        exit_signal = FindExit().find_exit(exit_strategy, order.entry, **kwargs)
        _entry_signal = next(pd.DataFrame(order.entry).itertuples())
        if exit_signal != (0,0,0):
            print("Exit signal found placing a exit order")
            close_order(order, exit_signal)
            order.exit_price, order.exit_time, order.pl_status = exit_signal
            order.exit_price = str(order.exit_price)
            order.save()
            return order
        else:
            print("no signal found :(!")
    


# start_exit_trading(SMAExitLevel1, stoploss=0.0005, target=0.0008, df=data)
# order_book = OrderBook()
# last_order = order_book.last_order()
# last_order.entry

In [83]:
od = connection.get_order(138129071)
# 138134425 , 138134403
td = od.get_associated_trade()
# dir(td)
# td.get_amount()
td.get_tradeId()

ValueError: No order with id 138129071

packet queue is empty, aborting
|ERROR|2022-06-09 06:07:42,416|packet queue is empty, aborting
|ERROR|2022-06-09 06:08:01,159|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Read timed out. (read timeout=5)
|ERROR|2022-06-09 06:08:13,043|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1654754883.0305798 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f1121ec1fd0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-09 06:08:27,468|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1654754897.46130

In [39]:
# order = connection.open_trade(
#                             symbol='EUR/USD',
#                             is_buy=True,
#                             amount=5,
#                             order_type="AtMarket",
# #                             order_range=2,
#                             rate=1.0526,
#                             limit=1.0537,
#                             is_in_pips=False,
#                             time_in_force='GTC',
#                         )
# order


In [8]:
# connection.is_subscribed('EUR/USD')
# od = Order.objects.first()
# od.orderId = order.get_orderId()
# od.tradeId = order.get_tradeId()
# od.save()
# Order.objects.filter(tradeId=None, pl_status=None)

# od = connection.get_order(138218585)


""


In [7]:
connection.subscribe_market_data('EUR/USD')

In [8]:
con.set_max_prices(200)

In [15]:
con.get_prices('EUR/USD')

,Bid,Ask,High,Low
2022-06-10 17:52:50.050,1.05241,1.05255,1.05042,1.06439
2022-06-10 17:52:50.381,1.05242,1.05256,1.05042,1.06439
2022-06-10 17:52:51.019,1.05243,1.05256,1.05042,1.06439
2022-06-10 17:52:51.790,1.05244,1.05256,1.05042,1.06439
2022-06-10 17:52:52.241,1.05243,1.05255,1.05042,1.06439
...,...,...,...,...
2022-06-10 17:54:46.817,1.05232,1.05245,1.05042,1.06439
2022-06-10 17:54:47.720,1.05231,1.05243,1.05042,1.06439
2022-06-10 17:54:49.557,1.05227,1.05240,1.05042,1.06439
2022-06-10 17:54:50.723,1.05227,1.05241,1.05042,1.06439


In [18]:
con.get_last_price('EUR/USD')

Bid     1.05222
Ask     1.05235
High    1.05042
Low     1.06439
Name: 2022-06-10 17:55:45.960000, dtype: float64

In [20]:
def create_order(**kwargs):
    last_row = connection.get_last_price('EUR/USD')
#     print(dict(last_row))
#     logger.debug(kwargs)
    if kwargs["is_buy"] == True:
        if kwargs["rate"] > last_row["Bid"]: 
            kwargs["order_type"] = "AtMarket"
            return connection.open_trade(**kwargs)
        else:
            return connection.create_entry_order(**kwargs)
    if kwargs["is_buy"] == False:
        if kwargs["rate"] < last_row["Bid"]:
            kwargs["order_type"] = "AtMarket"
            return connection.open_trade(**kwargs)
        else:
            return connection.create_entry_order(**kwargs)
        
data = {
        "symbol":'EUR/USD',
        "is_buy":True,
        "amount":5,
        "order_type":"Entry",
        "rate":1.0528,
        "limit":1.0538,
        "is_in_pips":False,
        "time_in_force":'GTC',
    }

create_order(**data)

In [4]:
import pandas as pd
def print_data(data, dataframe):
    print('%3d | %s | %s, %s, %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1], data['Rates'][2],
             data['Rates'][3]))

def arraySortedOrNot(arr):
 
    # Calculating length
    n = len(arr)
 
    # Array has one or no element or the
    # rest are already checked and approved.
    if n == 1 or n == 0:
        return True
 
    # Recursion applied till last element
    return arr[0] <= arr[1] and arraySortedOrNot(arr[1:])
# data.to_csv("feb.csv")

In [5]:
from icecream import ic
from datetime import datetime
import pandas_ta as pta
d1 = pd.read_csv("feb1.csv")
# d2 = pd.read_csv("dec2.csv")
# d3 = pd.read_csv("dec3.csv")
# d3 = pd.read_csv("second_jan.csv")
frames = [d1]
data = pd.concat(frames)
data.index = data.date.values

# # data = data.loc[(data["date"] > datetime.strptime("2022-01-05", "%Y-%m-%d"))]
data = data.drop(["date"], axis=1)
# data = data.iloc[40:3000] # 5831
data
# pd.DataFrame(index=data.date, data=data)


,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
2022-02-01 00:00:00,1.12286,1.12282,1.12288,1.12277,1.12299,1.12294,1.12300,1.12289,118
2022-02-01 00:01:00,1.12281,1.12284,1.12290,1.12271,1.12294,1.12297,1.12303,1.12284,127
2022-02-01 00:02:00,1.12285,1.12291,1.12298,1.12285,1.12297,1.12304,1.12311,1.12297,36
2022-02-01 00:03:00,1.12292,1.12287,1.12292,1.12283,1.12304,1.12299,1.12304,1.12297,40
2022-02-01 00:04:00,1.12287,1.12292,1.12293,1.12285,1.12300,1.12305,1.12306,1.12298,53
...,...,...,...,...,...,...,...,...,...
2022-02-09 23:56:00,1.14210,1.14212,1.14213,1.14209,1.14223,1.14224,1.14227,1.14222,22
2022-02-09 23:57:00,1.14210,1.14208,1.14210,1.14208,1.14224,1.14223,1.14224,1.14221,11
2022-02-09 23:58:00,1.14209,1.14209,1.14210,1.14208,1.14223,1.14222,1.14223,1.14221,12
2022-02-09 23:59:00,1.14210,1.14203,1.14210,1.14202,1.14223,1.14217,1.14223,1.14216,26


In [5]:
# import ta
# from ta.trend import *
# from ta.momentum import *
from IPython.display import display


# data["SMALL_SMA"]=sma_indicator(data.bidclose, window=7)
# data["LARGE_SMA"]=sma_indicator(data.bidclose, window=10)
# # Stochastic need to fix
# data["STOCH"] = stoch(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["STOCH_SIGNAL"] = stoch_signal(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["RSI"] = rsi(data.bidclose)
# data["SMA_SIGNAL"] = 

# len(data.index)
# data = kada

In [10]:
import importlib
from trading import strategy
importlib.reload(strategy)
from trading.strategy import BaseStrategy
import pandas as pd
import numpy as np
from ta.trend import *
from ta.momentum import *
import copy



class SMAStrategy(BaseStrategy):
    name = "smarsistochastic_strategy"
    description = """7,10 SMA, 7,3,3 Stochastic and RSI Strategy"""
    
    def signal_found(self, data:dict):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "SELL"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "BUY"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        
        df.loc[(df["sma_signal"] == "BUY")  & (df["stoch"] >= 60) & (df["stoch_signal"] >= 60), "stoch_signal_f"] = "CHUTIYA"
        df.loc[(df["sma_signal"] == "SELL")  & (df["stoch"] <= 40) & (df["stoch_signal"] <= 40), "stoch_signal_f"] = "CHUTIYA" 
        df["entry_signal"] = df.sma_signal
        df["entry_price"] = df.bidopen
        return self.signal_found(df.loc[(df["stoch_signal_f"]== "CHUTIYA")].tail(1))
        
    
    def apply_indicator(self):
        super(SMAStrategy, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["stoch"] = round(stoch(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["stoch_signal"] = round(stoch_signal(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 7)


    
# strategy = SMAStrategy()
# data[:220]
# # for i in range(51,100):
# strategy.dataframe = data[:222]
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# df = df.loc[(df["SMA_SIGNAL"]=="BUY") | (df["SMA_SIGNAL"]=="SELL")]
# dd

# dir(SMAStrategy)
SMAStrategy.get_target_price(1.04350, 0.0008)
# rs_client.set("ch", pickle.dumps(SMAStrategy))

1.0443

In [11]:
class SMAStrategy1(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "DOWN_CROSSOVER"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "UP_CROSSOVER"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        # df["entry_signal"] = np.select([(df["sma_signal"] == "UP_CROSSOVER"),(df["sma_signal"] == "DOWN_CROSSOVER")],["BUY","SELL"]) 
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        new_df = df[last_sma_crossover.name:df.iloc[-1].name]
        new_df = new_df.iloc[1: , :]
        signal = None
        data = None
        amt = 0.00003
        if last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            candle_counter = 0
            iter_counter = 0
            signal = "SELL"
            for row in new_df.itertuples():
                if row.max_bid+amt >= row.small_sma:
                    candle_counter = 0
                else:
                    candle_counter += 1
                if candle_counter >= 2 or iter_counter == 5:
                    break
                iter_counter += 1
            if candle_counter != 2:
                return None
            else:
                data = new_df[row.Index : row.Index]
                # ic(row)
            # sma_touched_df = new_df.head(2).loc[(new_df["max_bid"] >= new_df["small_sma"])]
            # if not sma_touched_df.empty:
            #     return None
            # signal = "SELL"
            # data = new_df.head(2).tail(1)
        elif last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            candle_counter = 0
            iter_counter = 0
            for row in new_df.itertuples():
                if row.min_bid-amt <= row.small_sma:
                    candle_counter = 0
                else:
                    candle_counter += 1
                if candle_counter >= 2 or iter_counter == 5:
                    break
                iter_counter += 1
            if candle_counter != 2:
                return None
            else:
                data = new_df[row.Index : row.Index]
            
            # sma_touched_df = new_df.head(2).loc[(new_df["min_bid"] <= new_df["small_sma"])]
            # if not sma_touched_df.empty:
            #     return None
            # signal = "BUY"
            # data = new_df.head(2).tail(1)
        if signal and not data.empty:
            # ic(signal)
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
        #     # ic(d)
            return d
        # ic(last_sma_crossover)
        return None
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        

# strategy = SMAStrategy1()
# strategy.dataframe = data #.iloc[120:187]
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).tail(50)
# dd

In [12]:
class SMAStrategy2(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name ]
        # new_df = new_df.iloc[: , :-1]
        signal = None
        data = pd.DataFrame()
        amt = 0.00003
        
        if second_last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            last_row = None
            for row in new_df.itertuples():
                if row.rsi_signal == "UP_REVERSE" and last_sma_crossover.sma_signal == "UP_CROSSOVER" and \
                    last_sma_crossover.min_bid > last_sma_crossover.small_sma:
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        elif second_last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "SELL"
            for row in new_df.itertuples():
                if row.rsi_signal == "DOWN_REVERSE" and last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and \
                    last_sma_crossover.max_bid < last_sma_crossover.small_sma:
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        if signal and not data.empty:
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
            return d
        return None
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        

# strategy = SMAStrategy2()
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).iloc[1650:1700]

In [13]:
class SMAStrategy3Exit(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""  
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        signal = self.entry_signal
        target = self.__class__.get_target_price(signal.entry_price, self.target, signal.entry_signal)
        stoploss = self.__class__.get_stoploss_price(signal.entry_price, self.stoploss, signal.entry_signal)
        df_last_date = df.tail(1).index[0]
        temp_df = df[:signal.Index][-10:]
        rows_min = min(temp_df.min_bid)
        rows_max = max(temp_df.max_bid)
        slp_price= 0.0002
        tgp_price = 0.0002
        new_df = df[signal.Index:df_last_date].iloc[1: , :][:100]
        exit_price = 0
        exit_time = None
        status = ""
        rsi_high_hit = 0
        rsi_low_hit = 0
        
        def last_prices_fetcher(price:float, storage:list):
            if len(storage) == 4:
                storage.append(price)
                storage.pop(0)
            else:
                storage.append(price)
            return storage
        
        def get_trend(price_list:list):
            avr = round(sum(price_list) / len(price_list), 5)
            # ic(price_list[-1] - avr)
            if price_list[-1] > avr:
                return "UP"
            elif price_list[-1] < avr:
                return "DOWN"
            else:
                return "SIDE"
        
        if signal.entry_signal == "BUY":
            last_row = None
            new_sl = min(signal.bidopen - slp_price, rows_min)
            new_tgp_row = None
            last_prices = []
            down_trend_counter = 0
            target_hit = False
            last_trend = ""
            for row in new_df.itertuples():
                last_prices_fetcher(row.bidclose, last_prices)
                trend = get_trend(last_prices)
                if last_row == None:
                    last_row = row
                if last_row.rsi > 70 and row.rsi < 70:
                    rsi_high_hit += 1
                if last_row.rsi < 30 and row.rsi > 30:
                    rsi_low_hit += 1
                
                # ic(last_row.rsi > 70 and row.rsi < 70)
                # ic(row.max_bid >= target)
                # ic(row)
                # ic(target)
                if new_tgp_row == None:
                    if row.max_bid >= signal.bidclose + tgp_price:
                        new_tgp_row = row  
                elif row.max_bid >= target:
                    target_hit = True
                    if row.rsi < 67:
                        exit_price = row.bidclose
                        exit_time = row.Index
                        status = "TG"
                        break
                elif row.min_bid <= new_sl:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SLPR"
                    break
                elif row.bidclose <= stoploss:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SL"
                    break
                # elif last_row.rsi > 71:
                #     tgp_broken = True
                # elif tgp_broken:
                #     if row.rsi < 50:
                #         exit_price = row.bidclose
                #         exit_time = row.Index
                #         status = "TGP"
                
                elif last_row.rsi > 70 and row.rsi < 70:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "TGP"
                    break
                last_row = row
                last_trend = trend
        elif signal.entry_signal == "SELL":
            last_row = None
            new_sl = max(rows_max, signal.bidopen + slp_price)
            new_tgp_row = None
            last_prices = []
            down_trend_counter = 0
            target_hit = False
            last_trend = ""
            for row in new_df.itertuples():
                last_prices_fetcher(row.bidclose, last_prices)
                trend = get_trend(last_prices)
                if last_row == None:
                    last_row = row
                if last_row.rsi > 30 and row.rsi < 30:
                    rsi_high_hit += 1
                if last_row.rsi < 70 and row.rsi > 70:
                    rsi_low_hit += 1
                if new_tgp_row == None:
                    if row.min_bid <= signal.bidclose - tgp_price:
                        new_tgp_row = row 
                elif row.min_bid <= target:
                    target_hit = True
                    if row.rsi > 33:
                        exit_price = row.bidclose
                        exit_time = row.Index
                        status = "TG"
                        break
                elif row.max_bid >= new_sl:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SLPR"
                    break
                elif row.bidclose >= stoploss:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SL"
                    break
                # elif last_row.rsi < 29:
                #     tgp_broken = True
                # elif tgp_broken:
                #     if row.rsi > 50:
                #         exit_price = row.bidclose
                #         exit_time = row.Index
                #         status = "TGP"
                elif last_row.rsi < 30 and row.rsi > 30:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "TGP"
                    break
                last_row = row
                last_trend = trend
        if all([exit_price, status, exit_time]):
            return exit_price, exit_time, status
        return 0,0,0

    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value
            
    

    
class SMAExitLevel1(BaseStrategy):
    
    def get_next_entry_signal(self):
        strategy = SMAStrategy3Level1(stoploss=self.stoploss, target=self.target)
        df = self.dataframe.copy()
        new_df = df[self.entry_signal.Index:]
        if new_df.empty:
            return None
        # this is temporary code for testing it need optimization
        for date_index in new_df.index:
            strategy.dataframe = df[:date_index]
            strategy.is_valid_dataframe()
            signal = strategy.get_signal()
            if signal:
                signal_df = pd.DataFrame(signal).iloc[0]
                if signal_df.name <= self.entry_signal.Index:
                    continue
                return signal
        return None
    
    def strategy(self):
        df = self.dataframe.copy()
        stoploss_price = self.__class__.get_stoploss_price(self.entry_signal.entry_price, self.stoploss, self.entry_signal.entry_signal)
        exit_strategy = SMAStrategy3Exit(target=self.target, stoploss=self.stoploss)
        exit_strategy.dataframe = df
        exit_strategy.entry_signal = self.entry_signal
        exit_strategy.is_valid_dataframe()
        exit_signal = exit_strategy.get_signal()
        
        # return exit_signal
        next_entry_signal = self.get_next_entry_signal()
        # ic(next_entry_signal)
        if next_entry_signal:
            next_entry_signal = pd.DataFrame(next_entry_signal).iloc[0]
        else:
            next_entry_signal = pd.DataFrame()
        
        if exit_signal == (0,0,0):
            new_df = self.dataframe[self.entry_signal.Index:]
            counter = 0
            for frame in new_df.itertuples():
                if self.entry_signal.entry_signal == "BUY" and frame.bidclose <= stoploss_price:
                    return frame.bidclose, frame.Index, "SL"
                elif self.entry_signal.entry_signal == "SELL" and frame.bidclose >= stoploss_price:
                    return frame.bidclose, frame.Index, "SL"
        elif exit_signal == (0,0,0) and not next_entry_signal.empty:
            next_entry_time = next_entry_signal.name
            signal_entry_time = self.entry_signal.Index
            if next_entry_time > signal_entry_time:
                if self.entry_signal.entry_signal == "SELL" and next_entry_signal.entry_signal == "SELL":
                    if self.entry_signal.bidclose - next_entry_signal.bidclose < -0.0002:
                        return next_entry_signal.bidclose, next_entry_time, "SLPR"
                elif self.entry_signal.entry_signal == "SELL" and next_entry_signal.entry_signal == "BUY":
#                     for time being exit all entry
                    return next_entry_signal.bidclose, next_entry_time, "TGP"
                elif self.entry_signal.entry_signal == "BUY" and next_entry_signal.entry_signal == "BUY":
                    if self.entry_signal.bidclose - next_entry_signal.bidclose > 0.0002:
                        return next_entry_signal.bidclose, next_entry_time, "SLPR"
                elif self.entry_signal.entry_signal == "BUY" and next_entry_signal.entry_signal == "SELL":
                # for time being exit and take new entry      
                    return next_entry_signal.bidclose, next_entry_time, "SLPR"
        return exit_signal


# SMAExitLevel1Copy = copy.deepcopy(SMAExitLevel1)
# d["entry_signal"] = {'2021-12-01 10:25:00': 'SELL'}

# 
# test_signal = next(pd.DataFrame(d).itertuples())
# tragedy = SMAExitLevel1(target=0.0010, stoploss=0.0005)
# kata = data
# print(kata.iloc[-1].name)
# tragedy.dataframe = kata
# tragedy.entry_signal = test_signal
# tragedy.is_valid_dataframe()
# dd= tragedy.get_signal()
# ic(test_signal)
# ic(dd)


In [17]:
ss = pd.DataFrame(sma_backtest.signal_data)
# from strategy import DefaultExitStrategy
# bd = ss.itertuples()
# p = next(bd)
# ic(p)
# for i in bd:
#     ic(i)
#     break
# for test_signal in ss.itertuples():
#     if pd.Timestamp(test_signal.Index) == pd.Timestamp("2021-12-01 01:21:00"):
#         break
# test_signal

def run_exit_strategy(exit_strategy, signal_df, main_df):
    exit_details = []
    exit_strategy = exit_strategy(stoploss=0.0005, target=0.0008)
    exit_strategy.dataframe = main_df
    signal_data = signal_df.itertuples()
    next(signal_data)
    exit_details.append((0,0,0))
    for signal in signal_data:
        exit_strategy.entry_signal = signal
        if exit_strategy.is_valid_dataframe():
            # if signal == test_signal:
            #     ic(signal)
            #     ic(exit_strategy.get_signal())
            exit_details.append(exit_strategy.get_signal())

    signal_df["exit_price"] = [detail[0] for detail in exit_details]
    signal_df["exit_time"] = [detail[1] for detail in exit_details]
    signal_df["status"] = [detail[2] for detail in exit_details]
    return signal_df
dp = run_exit_strategy(SMAExitLevel1, ss, test_data)
dp

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty,max_bid,...,small_sma,large_sma,rsi,rsi_signal,sma_signal,entry_price,entry_signal,exit_price,exit_time,status
2022-06-14 00:29:00,1.04203,1.04199,1.04206,1.04186,1.04215,1.04212,1.04219,1.04199,298,1.04206,...,1.041720,1.041627,68.474425,DOWN_REVERSE,,1.04199,SELL,0.00000,0,0
2022-06-14 01:17:00,1.04090,1.04115,1.04121,1.04081,1.04104,1.04129,1.04135,1.04096,162,1.04121,...,1.041211,1.041372,33.699023,UP_REVERSE,,1.04115,BUY,1.04157,2022-06-14 02:44:00,TGP
2022-06-14 01:20:00,1.04093,1.04105,1.04122,1.04093,1.04105,1.04118,1.04134,1.04105,227,1.04122,...,1.041063,1.041149,35.203868,UP_REVERSE,,1.04105,BUY,1.04068,2022-06-14 01:30:00,SLPR
2022-06-14 02:44:00,1.04174,1.04157,1.04177,1.04156,1.04189,1.04169,1.04191,1.04169,173,1.04177,...,1.041676,1.041611,61.693884,DOWN_REVERSE,,1.04157,SELL,1.04210,2022-06-14 03:05:00,SL
2022-06-14 03:08:00,1.04237,1.04213,1.04240,1.04196,1.04249,1.04225,1.04254,1.04210,216,1.04240,...,1.042087,1.041987,60.467382,DOWN_REVERSE,,1.04213,SELL,1.04277,2022-06-14 03:48:00,SLPR
2022-06-14 03:50:00,1.04280,1.04265,1.04280,1.04253,1.04292,1.04278,1.04292,1.04265,284,1.04280,...,1.042480,1.042397,66.636373,DOWN_REVERSE,,1.04265,SELL,1.04317,2022-06-14 04:01:00,SLPR
2022-06-14 04:05:00,1.04339,1.04321,1.04341,1.04316,1.04353,1.04336,1.04355,1.04332,336,1.04341,...,1.043117,1.042976,65.375817,DOWN_REVERSE,,1.04321,SELL,1.04244,2022-06-14 04:47:00,TG
2022-06-14 04:47:00,1.04237,1.04244,1.04244,1.04231,1.04250,1.04256,1.04256,1.04245,91,1.04244,...,1.042394,1.042429,36.946390,UP_REVERSE,,1.04244,BUY,1.04222,2022-06-14 05:02:00,SLPR
2022-06-14 05:32:00,1.04184,1.04197,1.04197,1.04182,1.04197,1.04209,1.04209,1.04195,99,1.04197,...,1.041881,1.041962,40.166502,UP_REVERSE,,1.04197,BUY,1.04308,2022-06-14 05:49:00,TG
2022-06-14 05:49:00,1.04324,1.04308,1.04333,1.04308,1.04337,1.04323,1.04347,1.04320,156,1.04333,...,1.043034,1.042952,66.570025,DOWN_REVERSE,,1.04308,SELL,1.04352,2022-06-14 06:23:00,SLPR


In [14]:

# CODE Block
class SMAStrategy3Level1(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    
    def signal_found(self, data):
        return data.to_dict()
    
    
    def create_signal(self, data, **kwargs):
        d_date = data.index[0]
        d = data.to_dict()
        d["entry_price"] = d["bidclose"]
        d["entry_signal"] = {d_date: kwargs.get("signal")}
        current_signal_df = pd.DataFrame(d).iloc[0]
        return d
    
    def strategy1(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name]
        new_df = new_df.iloc[::-1]
        signal = None
        data = pd.DataFrame()
        last_entry_signal = self.get_last_signal()
        last_entry_signal = next(pd.DataFrame(last_entry_signal).itertuples())
        tragedy = SMAStrategy3Exit(target=self.target, stoploss=self.stoploss)
        tragedy.dataframe = df
        tragedy.entry_signal = last_entry_signal
        tragedy.is_valid_dataframe()
        last_exit_signal= tragedy.get_signal()
        # ic(last_exit_signal)
        # ic(last_entry_signal.Index, last_entry_signal.bidclose)
        def get_trend(df):
            last_row_price = df.iloc[0].bidclose
            avr = round(df.iloc[:6].bidclose.sum() / 6, 5)
            if last_row_price > avr:
                return "UP"
            elif last_row_price < avr:
                return "DOWN"
            else:
                return "SIDE"
        
        if last_sma_crossover.rsi_signal == "UP_REVERSE" and not new_df.empty:
            signal = "BUY"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if max(row.bidopen, row.bidclose) < row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        elif last_sma_crossover.rsi_signal == "DOWN_REVERSE" and not new_df.empty:
            signal = "SELL"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if min(row.bidopen, row.bidclose) > row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        sma_crossover_counter = 0
        try:
            last_crossover = new_df.loc[(new_df["sma_signal_1"] == "UP_CROSSOVER") | (new_df["sma_signal_1"] == "DOWN_CROSSOVER")].iloc[-0]
        except:
            last_crossover = pd.DataFrame()
        for row in new_df.itertuples():
            if row.sma_signal_1.endswith("CROSSOVER"):
                break
            else:
                sma_crossover_counter += 1
                
        if not data.empty and signal == last_entry_signal.entry_signal:
            if "SLPR" in last_exit_signal:
                if (pd.Timestamp(data.index[0]) - pd.Timestamp(last_exit_signal[1])) / pd.Timedelta(minutes=1) < 13:
                    return None
                elif not last_crossover.empty:
                    if signal == "SELL" and last_crossover.sma_signal_1 == "UP_CROSSOVER" and sma_crossover_counter > 14:
                        return None
                    elif signal == "BUY" and last_crossover.sma_signal_1 == "DOWN_CROSSOVER" and sma_crossover_counter > 14:
                        return None
            elif last_exit_signal == (0,0,0):
                if (pd.Timestamp(data.index[0]) - pd.Timestamp(last_entry_signal.Index)) / pd.Timedelta(minutes=1) < 13:
                    return None

        if last_exit_signal[2] == "TG" and not data.empty:
            entry_diff = (pd.Timestamp(data.iloc[0].name) - pd.Timestamp(last_exit_signal[1])) / pd.Timedelta(minutes=1)
            if signal != last_entry_signal.entry_signal and entry_diff <= 15:
                ic(last_exit_signal)
                ic(last_entry_signal)
                return self.create_signal(data, signal=last_entry_signal.entry_signal)
        
        
        if not last_crossover.empty:
            if signal == "SELL" and last_crossover.sma_signal_1 == "UP_CROSSOVER" and sma_crossover_counter > 14:
                return None
            elif signal == "BUY" and last_crossover.sma_signal_1 == "DOWN_CROSSOVER" and sma_crossover_counter > 14:
                return None
        
        if signal and not data.empty:
            return self.create_signal(data, signal=signal)
        return None
       
    
    def get_previous_signal(self):
        negative_looper = list(range(-9,-1))[::-1]
        first_num = negative_looper[0]
        for n in negative_looper[1:]:
            signal = self.get_current_signal(last_signal_value=first_num, second_last_signal_value=n)
            if signal:
                return signal
            first_num = n
        return None

    
    def get_current_signal(self, **kwargs):
        """optimization required"""
        df = kwargs.get("dataframe", self.dataframe)
        last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[kwargs.get("last_signal_value", -1)]
        second_last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[kwargs.get("second_last_signal_value", -2)]
        new_df = df[:last_sma_crossover.name]
        new_df = new_df.iloc[::-1]
        signal = None
        data = pd.DataFrame()
        
        if last_sma_crossover.rsi_signal == "UP_REVERSE" and not new_df.empty:
            signal = "BUY"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if max(row.bidopen, row.bidclose) < row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        elif last_sma_crossover.rsi_signal == "DOWN_REVERSE" and not new_df.empty:
            signal = "SELL"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if min(row.bidopen, row.bidclose) > row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        
        if signal and not data.empty:
            return self.create_signal(data, signal=signal)
        return None
    
    def strategy(self, **kwargs):
        df = self.dataframe
        last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name]
        new_df = new_df.iloc[::-1]
        signal = None
        data = pd.DataFrame()
        
        if last_sma_crossover.rsi_signal == "UP_REVERSE" and not new_df.empty:
            signal = "BUY"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if max(row.bidopen, row.bidclose) < row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        elif last_sma_crossover.rsi_signal == "DOWN_REVERSE" and not new_df.empty:
            signal = "SELL"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if min(row.bidopen, row.bidclose) > row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        
        if signal and not data.empty:
            return self.create_signal(data, signal=signal)
        return None
        

    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
    
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        


class Level3(BaseStrategy):
        
    def get_current_signal(self, df, **kwargs):
        df1 = df.copy()
        df2 = df.copy()
        strategy = SMAStrategy3Level1(stoploss=self.stoploss, target=self.target)
        strategy.dataframe = df1
        strategy.is_valid_dataframe()
        current_signal = strategy.get_current_signal()
        previous_signal = strategy.get_previous_signal()
        if kwargs.get("need_previous_signal"):
            setattr(self, "_previous_signal", previous_signal)
        exit_strategy = SMAStrategy3Exit(stoplos=self.stoploss, target=self.target)
        exit_strategy.dataframe = df2
        exit_strategy.entry_signal = next(pd.DataFrame(previous_signal).itertuples())
        exit_strategy.is_valid_dataframe()
        last_exit_signal= exit_strategy.get_signal()
        # print(last_exit_signal, "exit signal for last entry siignal")
        # print(current_signal, "current signal")
        # print(previous_signal, "last signal")
        try:
            current_signal_df = next(pd.DataFrame(current_signal).itertuples())
        except:
            current_signal_df = pd.DataFrame()
            
        try:
            last_signal_df = next(pd.DataFrame(previous_signal).itertuples())
        except:
            last_signal_df = pd.DataFrame()
            
        if last_exit_signal and last_exit_signal[2] == "TG" and current_signal:
            entry_diff = (pd.Timestamp(current_signal_df.Index) - pd.Timestamp(last_exit_signal[1])) / pd.Timedelta(minutes=1)
            if current_signal_df.entry_signal != last_signal_df.entry_signal and entry_diff <= 15:
                sig_time, entry = list(*current_signal["entry_signal"].items())
                current_signal["entry_signal"] = {sig_time:last_signal_df.entry_signal}
                return current_signal
        if last_exit_signal == (0,0,0) and current_signal:
            last_entry_signal_price = last_signal_df.bidclose
            current_entry_signal_price = current_signal_df.bidclose
            diff = 0
            if last_signal_df.entry_signal == "BUY":
                diff = current_entry_signal_price - last_entry_signal_price
            else:
                diff = last_entry_signal_price - current_entry_signal_price
            
            if diff <= -0.0003:
                return None
            elif diff >= 0.0003:
                sig_time, entry = list(*current_signal["entry_signal"].items())
                current_signal["entry_signal"] = {sig_time:last_signal_df.entry_signal}
        return current_signal
    
    def strategy(self):
        df = self.dataframe.copy()
        current_signal = self.get_current_signal(df, need_previous_signal=True)
        # sig_time, entry = list(*self._previous_signal["entry_signal"].items())
        # previous_signal = self.get_current_signal(df[:sig_time])
        # print(previous_signal)
        # actual_previous_signal = 
        
        return current_signal
    
st = SMAStrategy3Level1(stoploss=0.0010, target=0.0005)
test_data = data
st.dataframe = test_data
ic(test_data.iloc[-1])
st.is_valid_dataframe()
d = st.get_signal()
d


ic| test_data.iloc[-1]: bidopen       1.04209
                        bidclose      1.04185
                        bidhigh       1.04219
                        bidlow        1.04176
                        askopen       1.04221
                        askclose      1.04197
                        askhigh       1.04232
                        asklow        1.04189
                        tickqty     389.00000
                        Name: 2022-06-14 15:29:00, dtype: float64


{'bidopen': {Timestamp('2022-06-14 14:59:00'): 1.04081},
 'bidclose': {Timestamp('2022-06-14 14:59:00'): 1.04096},
 'bidhigh': {Timestamp('2022-06-14 14:59:00'): 1.04106},
 'bidlow': {Timestamp('2022-06-14 14:59:00'): 1.04074},
 'askopen': {Timestamp('2022-06-14 14:59:00'): 1.04093},
 'askclose': {Timestamp('2022-06-14 14:59:00'): 1.04108},
 'askhigh': {Timestamp('2022-06-14 14:59:00'): 1.0412},
 'asklow': {Timestamp('2022-06-14 14:59:00'): 1.04087},
 'tickqty': {Timestamp('2022-06-14 14:59:00'): 708},
 'max_bid': {Timestamp('2022-06-14 14:59:00'): 1.04106},
 'min_bid': {Timestamp('2022-06-14 14:59:00'): 1.04074},
 'small_sma': {Timestamp('2022-06-14 14:59:00'): 1.0411514285714285},
 'large_sma': {Timestamp('2022-06-14 14:59:00'): 1.041195},
 'rsi': {Timestamp('2022-06-14 14:59:00'): 36.29394357862837},
 'rsi_signal': {Timestamp('2022-06-14 14:59:00'): 'UP_REVERSE'},
 'sma_signal': {Timestamp('2022-06-14 14:59:00'): ''},
 'entry_price': {Timestamp('2022-06-14 14:59:00'): 1.04096},
 'en

In [36]:
import datetime
dt = next(pd.DataFrame(d).itertuples())

# str(pd.Timestamp(dt.Index))

# if (datetime.datetime.strptime(dt.Index, '%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes=5)) < datetime.datetime.today():
#     print("RAJU")

'2022-02-09 23:45:00'

In [15]:
class SMAStrategy2Exit(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to7754w_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name ]
        # new_df = new_df.iloc[: , :-1]
        signal = None
        data = pd.DataFrame()
        amt = 0.00003
        
        if second_last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            last_row = None
            for row in new_df.itertuples():
                if row.rsi_signal == "UP_REVERSE" and last_sma_crossover.sma_signal == "UP_CROSSOVER":
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        elif second_last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "SELL"
            for row in new_df.itertuples():
                if row.rsi_signal == "DOWN_REVERSE" and last_sma_crossover.sma_signal == "DOWN_CROSSOVER":
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        # if signal and not data.empty:
        #     d_date = data.index[0]
        #     d = data.to_dict()
        #     d["entry_price"] = d["bidclose"]
        #     d["entry_signal"] = {d_date: signal}
        #     return d
        return df
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        

# strategy = SMAStrategy2()
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1)
# dd

In [16]:
test_data = data.copy() #[100:]
from trading import backtest
# from trading import strategy
# importlib.reload(strategy)
importlib.reload(backtest)
from trading.backtest import BackTestStrategy
# importlib.reload(strategy)
importlib.reload(backtest)

# exit_strategy=SMAExitLevel1,
sma_backtest = BackTestStrategy(SMAStrategy3Level1, test_data, target=0.0008,stoploss=0.0005, order_quantity=30000)
sma_backtest.start_backtesting()

  0%|          | 0/929 [00:00<?, ?it/s]

'backtesting has been completed, please generate report'

In [18]:
# import strategy
# importlib.reload(strategy)
# import backtest
# importlib.reload(backtest)
# kd = sma_backtest.get_report()
kd = sma_backtest.clean_report(dp)
# sma_backtest.signal_data
# kd["max_bid"] = kd[["bidopen","bidclose","bidlow","bidhigh"]].max(axis=1)2022-01-03 10:53:00
print(len(list(filter(lambda k : k == "SL", kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k == "TG", kd.status))) , "Total Target")
print(len(list(filter(lambda k : k == "TGP", kd.status))) , "Total TGP")
print(len(list(filter(lambda k : k == "SLP", kd.status))) , "Total SLP")
print(len(list(filter(lambda k : k == "SLPR", kd.status))) , "Total SLPR")
print(len(list(filter(lambda k : k in ["SL", "SLP"], kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k in ["TG","TGP"], kd.status))) , "Total Target")
print(f"Total Amount Earned {kd.pl.sum()}")
print(f"From Date: {kd.iloc[1].name}")
print(f"To Date: {kd.iloc[-1].name}")
kd = kd.drop(["askopen","askclose", "askhigh", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
        "rsi_signal","large_sma","small_sma","tickqty","pip"], axis=1)
print(kd.shape)
ic(kd[kd["status"] == "SLP"].pl.sum())
ic(kd[kd["status"] == "SLPR"].pl.sum())
ic(kd[kd["status"] == "TGP"].pl.sum())
ic(kd[kd["status"] == "SL"].pl.sum())
ic(kd[kd["status"] == "TG"].pl.sum())
print("False Signals")
print(len(kd[kd["status"] == 0].pl))
print("False Signals")

ic| kd[kd["status"] == "SLP"].pl.sum(): 0.0
ic| kd[kd["status"] == "SLPR"].pl.sum(): -212.99999999998988
ic| kd[kd["status"] == "TGP"].pl.sum(): 12.60000000000261
ic| kd[kd["status"] == "SL"].pl.sum(): -47.99999999998805
ic| kd[kd["status"] == "TG"].pl.sum(): 275.9999999999896


3 Total Stoploss
11 Total Target
1 Total TGP
0 Total SLP
14 Total SLPR
3 Total Stoploss
12 Total Target
Total Amount Earned 27.600000000014298
From Date: 2022-06-14 01:17:00
To Date: 2022-06-14 14:59:00
(30, 11)
False Signals
1
False Signals


In [19]:
# 1 Total Stoploss
# 22 Total Target
# 102 Total TGP
# 0 Total SLP
# 43 Total SLPR
# 1 Total Stoploss
# 124 Total Target
# Total Amount Earned 1339.7999999999379
# From Date: 2021-12-01 01:14:00
# To Date: 2021-12-03 18:42:00
# (204, 14)
# ic| kd[kd["status"] == "SLP"].pl.sum(): 0.0
# ic| kd[kd["status"] == "SLPR"].pl.sum(): -488.40000000002885
# ic| kd[kd["status"] == "TGP"].pl.sum(): 1152.8999999999724
# ic| kd[kd["status"] == "SL"].pl.sum(): -16.80000000000348
# ic| kd[kd["status"] == "TG"].pl.sum(): 692.0999999999977
# False Signals
# 36
# kd.to_csv("dec_test_half.csv")
kd.drop(["max_bid","min_bid","sma_signal"],axis=1).tail(50) #.to_csv("jan_report.csv") 
# diff = []
# for k in kd.itertuples():
#     try:
#         diff.append(k.exit_time - k.Index)
#     except:
#         diff.append(0)
# kd["diff"]= diff
# kd.drop(["max_bid","min_bid"],axis=1).head(50)
# kd[kd["status"] == "SLPR"].head(60).drop(["max_bid","min_bid","small_sma_8","large_sma_16","sma_signal_1","sma_signal"],axis=1)


,bidopen,rsi,entry_price,entry_signal,exit_price,exit_time,status,pl
2022-06-14 00:29:00,1.04203,68.474425,1.04199,SELL,0.00000,0,0,0.0
2022-06-14 01:17:00,1.04090,33.699023,1.04115,BUY,1.04157,2022-06-14 02:44:00,TGP,12.6
2022-06-14 01:20:00,1.04093,35.203868,1.04105,BUY,1.04068,2022-06-14 01:30:00,SLPR,-11.1
2022-06-14 02:44:00,1.04174,61.693884,1.04157,SELL,1.04210,2022-06-14 03:05:00,SL,-15.9
2022-06-14 03:08:00,1.04237,60.467382,1.04213,SELL,1.04277,2022-06-14 03:48:00,SLPR,-19.2
2022-06-14 03:50:00,1.04280,66.636373,1.04265,SELL,1.04317,2022-06-14 04:01:00,SLPR,-15.6
2022-06-14 04:05:00,1.04339,65.375817,1.04321,SELL,1.04244,2022-06-14 04:47:00,TG,23.1
2022-06-14 04:47:00,1.04237,36.946390,1.04244,BUY,1.04222,2022-06-14 05:02:00,SLPR,-6.6
2022-06-14 05:32:00,1.04184,40.166502,1.04197,BUY,1.04308,2022-06-14 05:49:00,TG,33.3
2022-06-14 05:49:00,1.04324,66.570025,1.04308,SELL,1.04352,2022-06-14 06:23:00,SLPR,-13.2


packet queue is empty, aborting
|ERROR|2022-06-14 16:47:38,514|packet queue is empty, aborting
|ERROR|2022-06-14 16:49:56,607|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Read timed out. (read timeout=5)
|ERROR|2022-06-14 16:50:06,532|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655225406.5191693 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778b5e0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-14 16:50:10,386|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655225410.38222

|ERROR|2022-06-14 16:51:49,511|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655225509.5077946 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778b580>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-14 16:51:59,792|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655225519.7856123 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778b490>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-14 16:52:04,385|Connection error:

|ERROR|2022-06-14 16:53:45,983|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655225625.9792533 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778b730>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-14 16:53:55,532|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655225635.5277827 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778b370>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-14 16:54:00,717|Connection error:

|ERROR|2022-06-14 16:55:29,237|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655225729.233518 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778b3d0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-14 16:56:18,532|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655225778.5150213 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778b700>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-14 16:56:23,600|Connection error: 

...

|ERROR|2022-06-14 21:46:15,152|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655243175.1489813 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778b400>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-14 21:46:19,702|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655243179.6986535 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778bb20>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-14 22:43:01,540|Connection error:

|ERROR|2022-06-14 23:12:53,469|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655248373.4675112 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778b460>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-14 23:12:58,654|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655248378.6533127 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778b8e0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-15 02:00:34,820|Connection error:

|ERROR|2022-06-15 05:26:14,330|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655270774.3264456 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778b9a0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-15 05:26:18,934|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655270778.930147 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778baf0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-15 10:26:51,540|Connection error: 

|ERROR|2022-06-15 11:42:04,268|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655293324.2635376 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778b970>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-15 11:42:09,573|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655293329.5698063 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778b640>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-15 11:47:07,042|Connection error:

|ERROR|2022-06-15 12:23:34,694|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655295814.6909132 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778bb20>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-15 12:23:40,122|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Max retries exceeded with url: /socket.io/?access_token=5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27&agent=pythonquants&wait_for_connection=False&transport=polling&EIO=4&t=1655295820.1185758 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f79d778bb50>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
|ERROR|2022-06-15 12:23:49,547|Connection error:

In [38]:
k = pd.Timestamp(test_signal.Index) - pd.Timedelta(250, "m")
# data[str(k):test_signal.Index]
k < pd.Timestamp(test_signal.Index)

True

In [16]:
kd = sma_backtest.clean_report(dp)[["bidopen","bidclose","entry_price","entry_signal","exit_price","exit_time","status","pl"]]
print(len(list(filter(lambda k : k == "SL", kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k == "TG", kd.status))) , "Total Target")
print(len(list(filter(lambda k : k == "TGP", kd.status))) , "Total TGP")
print(len(list(filter(lambda k : k == "SLP", kd.status))) , "Total SLP")
print(len(list(filter(lambda k : k == "SLPR", kd.status))) , "Total SLPR")
print(len(list(filter(lambda k : k in ["SL", "SLP"], kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k in ["TG","TGP"], kd.status))) , "Total Target")
print(f"Total Amount Earned {kd.pl.sum()}")
print(f"From Date: {kd.iloc[1].name}")
print(f"To Date: {kd.iloc[-1].name}")
# kd = kd.drop(["askopen","askclose", "askhigh", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         "rsi_signal","large_sma","small_sma","tickqty","pip"], axis=1)
print(kd.shape)
ic(kd[kd["status"] == "SLP"].pl.sum())
ic(kd[kd["status"] == "SLPR"].pl.sum())
ic(kd[kd["status"] == "TGP"].pl.sum())
ic(kd[kd["status"] == "SL"].pl.sum())
ic(kd[kd["status"] == "TG"].pl.sum())
print("False Signals")
len(kd[kd["status"] == 0].pl)

ic| kd[kd["status"] == "SLP"].pl.sum(): 0.0
ic| kd[kd["status"] == "SLPR"]

11 Total Stoploss
208 Total Target
125 Total TGP
0 Total SLP
181 Total SLPR
11 Total Stoploss
333 Total Target
Total Amount Earned 3797.6999999999725
From Date: 2022-05-01 22:28:00
To Date: 2022-05-13 20:51:00
(586, 8)


.pl.sum(): -2374.799999999997
ic| kd[kd["status"] == "TGP"].pl.sum(): 1334.3999999999621
ic| kd[kd["status"] == "SL"].pl.sum(): -195.30000000000715
ic| kd[kd["status"] == "TG"].pl.sum(): 5033.400000000014


False Signals


61

In [17]:
# dp[["exit_time", "entry_signal", "pl","entry_price","exit_price","status"]].to_csv("may2022_filter_str.csv")

# Backtesting Packages

In [160]:
from tqdm.notebook import trange, tqdm
import time

# Backtesting Code

## Settings

In [161]:
entryStrategy = None
exitStrategy = None
stoploss = None
target = None
quantity = 10000
backtesting_dataframe = pd.DataFrame()

#### Start Backtesting Code

In [171]:
# entryStrategy = entryStrategy(stoploss=stoploss, target=target)

# for row_number in trange(1,len(backtesting_dataframe)):
    

In [13]:
"Last candle updated of time frame %s" % data.iloc[-1].name
# '%s:443/%s' % (connection.trading_url, "trading/close_trade")

'Last candle updated of time frame 2022-02-10 00:00:00'

NameError: name 'df' is not defined